In [2]:
import pandas as pd
import numpy as np

In [3]:
df_product = pd.read_csv('Assets/data/Hackathon_DimProduct_SAN_vShared.csv',header=0)
df_product

,ProductKey,BrandKey,SupplierKey,ProductCategory_Lvl1,ProductCategory_Lvl2
0,key_52138,3521,1039040101,Category A,category aa
1,19864,3521,1039040125,Category A,Category AC
2,95144,1713,1049688101,Category A,Category AA
3,23070,3521,1050235101,Category A,Category AC
4,93165,3521,1039040125,Category A,Category AC
...,...,...,...,...,...
293,12353,3521,1036290101,Category A,Category AB
294,55442,132,1036290101,Category A,Category AC
295,78206,3521,1039040101,Category A,Category AC
296,55438,132,1036290101,Category A,Category AB


In [44]:
df_product.dtypes

ProductKey              object
BrandKey                 int64
SupplierKey              int64
ProductCategory_Lvl1    object
ProductCategory_Lvl2    object
dtype: object

In [52]:
import string
import re

def remove_punctuations(ts):
    '''Takes a word and removes all punctuations.
     :parameter
     ts: str
        the word to be cleaned
    :returns
    -----
    str
        the cleaned version
      '''
    ts=str(ts)
    for punctuation in string.punctuation:
        ts = ts.replace(punctuation, '')
    return ts

def remove_non_numbers(ts):
    '''Takes a word and removes all non-digit characters.
     :parameter
     ts: str
        the word to be cleaned
    :returns
    -----
    str
        the cleaned version
      '''
    return int(re.sub('[^0-9.]', '',str(ts)))

def name_corrector(ts):
    '''Takes a word and removes all whitespaces, transforms it to lowercase,
     and connects the words using '_'. Also checks for misspelling.
     :parameter
     ts: str
        the word to be corrected
    :returns
    -----
    str
        the corrected version
      '''
    return re.sub('categrory','category','_'.join(re.sub(' +',' ',ts).lower().split(' ')))

In [45]:
df_product.columns

Index(['ProductKey', 'BrandKey', 'SupplierKey', 'ProductCategory_Lvl1',
       'ProductCategory_Lvl2'],
      dtype='object')

In [51]:
df_product_clean = df_product.copy()
for col in ['ProductKey', 'BrandKey', 'SupplierKey']:
    df_product_clean[col] = df_product[col].apply(remove_punctuations)
    df_product_clean[col] = df_product[col].apply(remove_non_numbers)

for col in ['ProductCategory_Lvl1','ProductCategory_Lvl2']:
    df_product_clean[col]= df_product[col].apply(remove_punctuations)
    df_product_clean[col] = df_product[col].apply(name_corrector)


In [ ]:
import numpy as np
from nltk.corpus import stopwords
import spacy
nlp = spacy.load('it_core_news_sm')
import string
import re
import gensim
from gensim.utils import simple_preprocess
class preprocess_pip():
    def __init__(self,df ,column_name):
        self.df = df
        self.old_text = column_name
        self.text = column_name+'_clean'

    def apply_preprocessing(self):
#         self.df[self.text] =  self.df[self.old_text].astype('string')
        self.df[self.text] = self.df[self.old_text].apply(self.na_filter)
        self.df.dropna(subset=[self.text],inplace=True)
        self.df[self.text] = self.df[self.text].apply(self.remove_punctuations)
        self.df[self.text] = self.df[self.text].apply(self.to_lower)
        self.df[self.text] = self.df[self.text].apply(self.stop_word_elm)
        self.df[self.text] = self.df[self.text].apply(self.deEmojify)
        self.df[self.text] = self.df[self.text].apply(self.na_filter)
        self.df.dropna(axis=0, how='any',subset=[self.text])
#         self.df.drop('index',axis=1,inplace=True)
        self.df.reset_index(inplace=True)
        self.df.drop('index',axis=1,inplace=True)
        self.df[self.text].replace("[0-9,€$-;\'\"]", '', regex=True,inplace=True)
        self.df.dropna(subset=[self.text],inplace=True)
        self.df=self.df[self.df[self.text]!='']


    def na_filter(self, text):
        if len(text)>0:
            return text
        return

    def remove_punctuations(self, text):
        text=str(text)
        for punctuation in string.punctuation:
            text = text.replace(punctuation, '')
        return text

    def to_lower(self, text):
        return text.lower()

    def stop_word_elm(self, text):
        new_text=[]
        for i in text.split():
            if i not in italian_stopwords:
                new_text.append(i)
        return ' '.join(new_text)

    def deEmojify(self, text):
        regrex_pattern = re.compile(pattern = "["
            u"\U0001F600-\U0001F64F"  # emoticons
            "🤮"
            "🤬"
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags = re.UNICODE)
        return regrex_pattern.sub(r'',text)

    def sent_to_words(self, text):
        for sentence in text:
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations